In [ ]:
import torch
import torch.nn as nn

x = torch.rand((2, 1), requires_grad=False)
y = torch.rand((2, 1))

f = torch.sum(x + y)
x.requires_grad = True
f.backward()

In [ ]:
6 << 1

# Torch

### buffer

In [ ]:
import torch

mdl = torch.nn.Linear(1, 1)
mdl.register_buffer("free_node", torch.rand((2, 1)))

In [ ]:
for buf in mdl.buffers():
    print(id(buf))

In [ ]:
mdl.bias

## pytest

In [ ]:
import ipytest

ipytest.autoconfig()